# EDA

## 1. 개발 환경 구축 및 데이터 로드 (Ground Zero)
> #### **데이터 다운로드 전 필수로 해야하는 일**

👉 **EDA의 첫 단추는 **모든 팀원이 오류 없이 데이터를 읽어올 수 있는 상태**를 만드는 것입니다.**

👉 우리 팀원 5명이 각자의 로컬 `/data/raw/` 폴더에 압축을 푼 `sprint_ai_project1_data` 폴더를 넣어두었다고 가정하겠습니다.

### 1.1 데이터 경로 설정

In [46]:
import pandas as pd
import json
import os
from glob import glob

# [Step 1-1] 현재 위치 기준 데이터 최상위 경로 설정
# '..'은 부모 폴더(pjt-sprint_AI07...)로 올라간다는 뜻입니다.
base_path = os.path.join('..', 'data', 'raw', 'sprint_ai_project1_data')
json_pattern = os.path.join(base_path, 'train_annotations', '*', '*', '*.json')

# [Step 1-2] 모든 JSON 파일 경로 리스트 만들기
# glob은 '유리구슬(glob)'처럼 조건에 맞는 모든 파일을 찾아주는 아주 기특한 도구입니다.
json_files = glob(json_pattern)
print(f"✅ 찾은 JSON 파일 개수: {len(json_files)}개")

✅ 찾은 JSON 파일 개수: 763개


### 문제 : annotations 데이터 폴더의 복잡한 구조
> Kaggle에서 데이터 구조를 살펴보면, `train_images`, `test_images` 폴더는 매우 단순한 반면,
> `train_annotations` 데이터는 상당히 깊고 복잡한 구조를 가졌다.

> ### 폴더명으로 파싱의 힌트를 얻었다(규칙성 존재)
- 이가 없으면 잇몸으로, 하나하나 까본다.
- **참고** 사진 1장에 알약이 4개가 배치된 데이터도 있고, 3개가 배치된 데이터도 존재했다.

    ```powershell
    🗂️train_annotaionts  
    │
    ├── 🗂️K-001900-016548-019607-029451_json   --> 사진 1장에 알약이 4개라면 6자리의 숫자코드가 4개
    │   │
    │   │   --> K-[1번째 약넘버]-[2번째 약넘버]-[3번째 약넘버]-[4번째 약넘버]_json  -> 제목이 나타내는 정보   
    │   │
    │   └── 🗂️K-001900 --> 1번째 약넘버(.json 내부에서 "drug_N: 알약코드" 혹은 "dl_mapping_code: 제품코드"에 표기됨)
    │       │
    │       ├──📄K-001900-016548-019607-029451_0_2_0_2_70_000_200.json  
    │       ├──📄K-001900-016548-019607-029451_0_2_0_2_75_000_200.json  
    │       └──📄K-001900-016548-019607-029451_0_2_0_2_90_000_200.json  
    │
    │           --> 원─핫 인코딩 된 클래스 같음 (추정)
    │    
    │           --> 0_2_0_2 (불확실) : 0 = "drug_S: 알약상태（정상）"
    │                                 2 = "back_color: 촬영배경（연회색 배경）",
    │                                 0 = "drug_dir: 알약방향（앞면）"
    │                                 2 = "light_color: 촬영조명（주백색）"
    │    
    │           --> (70,75,95)_000_200 (확실) : 70,75,95 = "camera_la: 카메라 위도"
    │                                               000 = "camera_la: 카메라 경도"
    │                                               200 = "size: 이미지 사이즈"
    │............  
    ```


#### 1.1.1 json 데이터 샘플 구조 확인(1)

In [47]:
if json_files:
    with open(json_files[0], 'r', encoding='utf-8') as f:
        sample_data = json.load(f)
        print("\n🔍 JSON 데이터 샘플 1번째 구조:")
        print(json.dumps(sample_data, indent=4, ensure_ascii=False)[:]) # [:50] -> 앞부분만 살짝 보기
else:
    print("❌ 파일을 찾지 못했습니다. 경로를 다시 확인해주세요!")


🔍 JSON 데이터 샘플 1번째 구조:
{
    "images": [
        {
            "file_name": "K-001900-016548-019607-029451_0_2_0_2_70_000_200.png",
            "width": 976,
            "height": 1280,
            "imgfile": "K-001900-016548-019607-029451_0_2_0_2_70_000_200.png",
            "drug_N": "K-001900",
            "drug_S": "정상알약",
            "back_color": "연회색 배경",
            "drug_dir": "앞면",
            "light_color": "주백색",
            "camera_la": 70,
            "camera_lo": 0,
            "size": 200,
            "dl_idx": "1900",
            "dl_mapping_code": "K-001900",
            "dl_name": "보령부스파정 5mg",
            "dl_name_en": "Buspar Tab. 5mg Boryung",
            "img_key": "http://connectdi.com/design/img/drug/1Mxwka5v0lL.jpg",
            "dl_material": "부스피론염산염",
            "dl_material_en": "Buspirone Hydrochloride",
            "dl_custom_shape": "정제, 저작정",
            "dl_company": "보령제약(주)",
            "dl_company_en": "Boryung",
            "di_company_mf": "",


> #### json 데이터 분석

#### 1.1.2 json 데이터 샘플 구조 확인(2)

In [49]:
if json_files:
    with open(json_files[762], 'r', encoding='utf-8') as f:
        sample_data = json.load(f)
        print("\n🔍 JSON 데이터 샘플 2번째 구조:")
        print(json.dumps(sample_data, indent=4, ensure_ascii=False)[:]) # [:50] -> 앞부분만 살짝 보기
else:
    print("❌ 파일을 찾지 못했습니다. 경로를 다시 확인해주세요!")


🔍 JSON 데이터 샘플 2번째 구조:
{
    "images": [
        {
            "file_name": "K-003544-004543-016548-027993_0_2_0_2_90_000_200.png",
            "width": 976,
            "height": 1280,
            "imgfile": "K-003544-004543-016548-027993_0_2_0_2_90_000_200.png",
            "drug_N": "K-027993",
            "drug_S": "정상알약",
            "back_color": "연회색 배경",
            "drug_dir": "앞면",
            "light_color": "주백색",
            "camera_la": 90,
            "camera_lo": 0,
            "size": 200,
            "dl_idx": "27993",
            "dl_mapping_code": "K-027993",
            "dl_name": "졸로푸트정 100mg",
            "dl_name_en": "Zoloft Tab. 100mg",
            "img_key": "http://connectdi.com/design/img/drug/154599025952100024.jpg",
            "dl_material": "설트랄린염산염",
            "dl_material_en": "Sertraline Hydrochloride",
            "dl_custom_shape": "정제, 저작정",
            "dl_company": "한국화이자제약(주)",
            "dl_company_en": "Pfizer Korea",
            "di_compa